# Unsupervised Domain Adaptation Project


## 1: Data download
Load data to project from Google Drive. Copy a subset of classes of images to the path:
- `adaptiope_small/product_images`
- `adaptiope_small/real_life` 

two directories. They represent images from two different domain **product** and **real_life**

In [1]:
from os import makedirs, listdir
from tqdm import tqdm
from google.colab import drive
from os.path import join
from shutil import copytree

drive.mount('/content/gdrive')

!mkdir dataset
!cp "gdrive/My Drive/Colab Notebooks/data/Adaptiope.zip" dataset/
# !ls dataset

!unzip -qq dataset/Adaptiope.zip   # unzip file

!rm -rf dataset/Adaptiope.zip 
!rm -rf adaptiope_small

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
mkdir: cannot create directory ‘dataset’: File exists
replace Adaptiope/product_images/watering can/watering can_080.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [2]:
!mkdir adaptiope_small
classes = listdir("Adaptiope/product_images")
print(classes)
classes = ["backpack", "bookcase", "car jack", "comb", "crown", "file cabinet", "flat iron", "game controller", "glasses",
           "helicopter", "ice skates", "letter tray", "monitor", "mug", "network switch", "over-ear headphones", "pen",
           "purse", "stand mixer", "stroller"]
domain_classes = ["product_images", "real_life"]
for d, td in zip(["Adaptiope/product_images", "Adaptiope/real_life"], ["adaptiope_small/product_images", "adaptiope_small/real_life"]):
  makedirs(td)
  for c in tqdm(classes):
    c_path = join(d, c)
    c_target = join(td, c)
    copytree(c_path, c_target)

['ice skates', 'chainsaw', 'hot glue gun', 'grill', 'tank', 'computer', 'coat hanger', 'speakers', 'sleeping bag', 'razor', 'skeleton', 'syringe', 'bookcase', 'car jack', 'boxing gloves', 'microwave', 'smartphone', 'toothbrush', 'stapler', 'smoking pipe', 'wheelchair', 'ring binder', 'rc car', 'projector', 'in-ear headphones', 'hourglass', 'bicycle', 'golf club', 'compass', 'sword', 'hoverboard', 'fighter jet', 'corkscrew', 'acoustic guitar', 'flat iron', 'spatula', 'mixing console', 'cellphone', 'comb', 'lawn mower', 'hat', 'motorbike helmet', 'ice cube tray', 'helicopter', 'binoculars', 'pen', 'watering can', 'vr goggles', 'computer mouse', 'letter tray', 'hard-wired fixed phone', 'puncher', 'scissors', 'power strip', 'mug', 'pogo stick', 'roller skates', 'network switch', 'bottle', 'stand mixer', 'webcam', 'phonograph', 'brachiosaurus', 'keyboard', 'knife', 'fan', 'office chair', 'rubber boat', 'stroller', 'trash can', 'monitor', 'bicycle helmet', 'scooter', 'diving fins', 'ruler', 

100%|██████████| 20/20 [00:05<00:00,  3.92it/s]


In [3]:
product_path = 'adaptiope_small/product_images'
real_life_path = 'adaptiope_small/real_life'

## 2: Domain-Adversarial training of Neural Network
We implement DANN UDA method [DANN](https://arxiv.org/pdf/1505.07818.pdf)

 

### 2.0: Import Libraries and Data Loading


In [4]:
from PIL import Image
from os.path import join
import math

img = Image.open(join(product_path, 'backpack', 'backpack_003.jpg'))
print('Image size: ', img.size)
#img

Image size:  (679, 679)


import libraries

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import softmax
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import vgg11, alexnet 
from torch.utils.data import DataLoader, random_split
from torchvision.transforms.transforms import ToTensor

configuration constants

In [6]:
img_size = 256
# mean, std used by pre-trained models from PyTorch
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
config = dict(epochs=10, batch_size=64,lr=0.01, wd=0.001, momentum=0.9, alpha=10, beta=0.75, gamma=10)

Configue GPU

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [8]:
def get_dataset(root_path):
  '''
    Get dataset from specific data path

    # parameters:
        root_path: path to image folder

    # return: train_loader, test_loader
  '''
  # Construct image transform
  image_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
  ])

  # Load data from filesystem
  image_dataset = ImageFolder(root_path, transform=image_transform)

  return image_dataset

def get_dataloader(dataset, batch_size, shuffle_train=True, shuffle_test=False):
  '''
    Get DataLoader from specific data path

    # parameters:
        dataset: ImageFolder instance
        batch_size: batch_size for DataLoader
        shuffle_train: whether to shuffle training data
        shuffle_test: whether to shuffle test data
  '''
  # Get train, test number
  num_total = len(dataset)
  num_train = int(num_total * 0.8 + 1)
  num_test  = num_total - num_train

  # random split dataset
  data_train, data_test = random_split(dataset, [num_train, num_test])

  # initialize dataloaders
  loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=shuffle_train)
  loader_test  = DataLoader(data_test, batch_size=batch_size, shuffle=shuffle_test)

  return loader_train, loader_test

### 2.1 Define Feature Extractor with Pretrain Network

In [9]:
class FeatureExtractor(nn.Module):
  def __init__(self):
    super(FeatureExtractor, self).__init__()

    # Feature Extractor with AlexNet
    self.feature_extractor = alexnet(pretrained=True)
    self.feature_dim = self.feature_extractor.classifier[-1].in_features

    # make the last layer identity
    self.feature_extractor.classifier[-1] = nn.Identity()

  def forward(self, x):
    return self.feature_extractor(x)
  
  def output_dim(self):
    return self.feature_dim

### 2.2 Define Classifier, Discriminator with RevereLayerF for training the Feature Extractor

In [10]:
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, output_dim)
        )
    
    def forward(self, X):
        return self.classifier(X) 

In [11]:
from torch.autograd import Function

class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, tensor):
        return tensor.view_as(tensor)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg(), None

In [12]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.discriminator =  nn.Sequential(
            nn.Linear(int(input_dim), 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024,1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        validity = self.discriminator(x)
        return validity 

In [13]:
class DANN(nn.Module):
  # def __init__(self, num_classes, adversarial=True):
  def __init__(self, num_classes):
    super(DANN, self).__init__()
    self.output_dim = num_classes

    # define inner network component
    self.feature_extractor = FeatureExtractor()
    self.classifier = Classifier(self.feature_extractor.output_dim(), num_classes)
    self.discriminator = Discriminator(self.feature_extractor.output_dim())  
  
  def forward(self, x):
    feature_output = self.feature_extractor(x)

    class_pred = self.classifier(feature_output)

    # Add a ReverseLayer here for negative gradient computation
    reverse_feature = ReverseLayerF.apply(feature_output)
    domain_pred = self.discriminator(reverse_feature)

    return class_pred, domain_pred 

### 2.3 Cost function

In [14]:
def get_class_loss_func():
  return nn.CrossEntropyLoss()

### 2.4 Optimizer

Setting the **learning rate** according to the original [paper](https://arxiv.org/pdf/1505.07818.pdf) section 5.2.2

$$ \mu_p =  \frac{\mu_0}{(1+\alpha \cdot p)^\beta}$$

where p is the training progress linearly changing from 0 to 1.

In [15]:
def get_optimizer(model, config, progress, adversarial=True):
  '''
  Config Optimizer
  '''
  learning_rate = config['lr']
  learning_rate = learning_rate / ((1 + config['alpha']*progress)**config['beta'])

  weight_decay  = config['wd']
  momentum      = config['momentum']

  feature_ext   = model.get_submodule("feature_extractor")
  classifier    = model.get_submodule("classifier")
  discriminator = model.get_submodule("discriminator")

  pre_trained_weights = feature_ext.parameters()

  if adversarial:
    other_weights = list(classifier.parameters()) + list(discriminator.parameters())
  else:
    other_weights = list(classifier.parameters())

  # assign parameters to parameters
  optimizer = torch.optim.SGD([
    {'params': pre_trained_weights},
    {'params': other_weights, 'lr': learning_rate}
  ], lr= learning_rate/10, weight_decay=weight_decay, momentum=momentum)
  
  return optimizer

### 2.5 Training Loop and Testing Loop

In [16]:
def train_loop(dataloader, model, device, progress):
  """
    Return:
      @best_state: best performance model state parameters
      @best_loss: best performance loss
  """
  size = len(dataloader.dataset)
  loss_fn = get_class_loss_func()

  optimizer = get_optimizer(model, config, progress, adversarial=False)

  best_loss  = float('inf')
  best_state = None

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    
    # compute prediction and loss
    class_pred, _ = model(X)

    # classification loss
    loss = loss_fn(class_pred, y)
    
    # store best state
    curr_loss = loss.item()
    if curr_loss < best_loss:
      best_loss  = curr_loss
      best_state = model.state_dict()
      print(f"## Update ## best_state with loss: {curr_loss:>7f}")
    
    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 10 == 0:
      current = batch * len(X)
      print(f"## Meter  ## current loss: {curr_loss:>7f} [{current:>5d}/{size:>5d}]")
    
  return best_state, best_loss

In [17]:
def test_loop(dataloader, model, device):
  test_loss, correct = 0, 0
  loss_fn = get_class_loss_func()

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      class_pred, _ = model(X)

      test_loss += loss_fn(class_pred, y).item()
      correct += (class_pred.argmax(1) == y).type(torch.float).sum().item()

  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

  return test_loss, correct

### 2.6 Training Function

In [18]:
def training(model, train_dataloader, test_dataloader, config, device):
  epochs = config['epochs']
  # print(f"Learning_rate {config['lr']}, weight_decay {config['wd']}")

  best_state, best_loss = None, float('inf')
  no_improve_count = 0

  for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n------------------")
    progress = epoch/epochs

    curr_state, _ = train_loop(train_dataloader, model, device, progress)

    # Test with curr_state
    model.load_state_dict(curr_state)

    test_loss, _ = test_loop(test_dataloader, model, device)

    # store the best performed state parameters
    if test_loss < best_loss:
      no_improve_count = 0

      best_state = curr_state
      best_loss  = test_loss
    else:
      no_improve_count += 1
    
    if no_improve_count > 2:
      print(f"## No Improvement on Test Set, Stopping ##")
      break

  model.load_state_dict(best_state)
  print("Done")


## 3 Training without using Domain Adaptation techniques

 ### 3.1 Product Domain -> Real Life

In [19]:
# Get dataloader
product_dataset   = get_dataset(product_path)
real_life_dataset = get_dataset(real_life_path)

#### 3.1.1 Training on Source Domain

In [20]:
train_dataloader, test_dataloader = get_dataloader(product_dataset, config['batch_size'])

model = DANN(len(product_dataset.classes)).to(device)

# Training
training(model, train_dataloader, test_dataloader, config, device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1
------------------
## Update ## best_state with loss: 3.000999
## Meter  ## current loss: 3.000999 [    0/ 1601]
## Update ## best_state with loss: 2.959620
## Update ## best_state with loss: 2.926028
## Update ## best_state with loss: 2.904498
## Update ## best_state with loss: 2.808029
## Update ## best_state with loss: 2.693931
## Update ## best_state with loss: 2.628657
## Update ## best_state with loss: 2.576085
## Update ## best_state with loss: 2.549546
## Update ## best_state with loss: 2.382282
## Meter  ## current loss: 2.382282 [  640/ 1601]
## Update ## best_state with loss: 2.195917
## Update ## best_state with loss: 1.951670
## Update ## best_state with loss: 1.903493
## Update ## best_state with loss: 1.634728
## Update ## best_state with loss: 1.402812
## Update ## best_state with loss: 1.268612
## Update ## best_state with loss: 0.888332
## Update ## best_state with loss: 0.813600
## Update ## best_state with loss: 0.652161
## Meter  ## current loss: 0.733628 [

#### 3.1.2 Test on Real Life

In [21]:
loader_target_dataset = DataLoader(real_life_dataset, batch_size=config['batch_size'], shuffle=False)

# model.load_state_dict(torch.load('model_state.pt', map_location='cpu'))
test_loop(loader_target_dataset, model, device)

Test Error: 
 Accuracy: 64.6%, Avg loss: 1.221270 



(1.2212704503908753, 0.646)

In [22]:
del train_dataloader, test_dataloader, loader_target_dataset
del model
print(torch.cuda.memory_allocated())

513480192


### 3.2 Real Life -> Product


#### 3.2.1 Training on Real Life

In [23]:
train_dataloader, test_dataloader = get_dataloader(real_life_dataset, config['batch_size'])

model = DANN(len(real_life_dataset.classes)).to(device)

# Training
training(model, train_dataloader, test_dataloader, config, device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1
------------------
## Update ## best_state with loss: 3.019980
## Meter  ## current loss: 3.019980 [    0/ 1601]
## Update ## best_state with loss: 2.985003
## Update ## best_state with loss: 2.971178
## Update ## best_state with loss: 2.941204
## Update ## best_state with loss: 2.902153
## Update ## best_state with loss: 2.869621
## Update ## best_state with loss: 2.761132
## Update ## best_state with loss: 2.707194
## Meter  ## current loss: 2.707194 [  640/ 1601]
## Update ## best_state with loss: 2.656637
## Update ## best_state with loss: 2.635004
## Update ## best_state with loss: 2.562607
## Update ## best_state with loss: 2.362159
## Update ## best_state with loss: 2.349334
## Update ## best_state with loss: 2.225987
## Update ## best_state with loss: 1.874920
## Update ## best_state with loss: 1.710216
## Meter  ## current loss: 1.710216 [ 1280/ 1601]
## Update ## best_state with loss: 1.480511
## Update ## best_state with loss: 1.214254
Test Error: 
 Accuracy: 33.3%, 

#### 3.2.2 Testing on Product


In [24]:
loader_target_dataset = DataLoader(product_dataset, batch_size=config['batch_size'], shuffle=False)

# model.load_state_dict(torch.load('model_state.pt', map_location='cpu'))
test_loop(loader_target_dataset, model, device)

Test Error: 
 Accuracy: 79.2%, Avg loss: 0.701823 



(0.701822874834761, 0.792)

In [25]:
del train_dataloader, test_dataloader, loader_target_dataset
del model
print(torch.cuda.memory_allocated())

513021440


## 4: Define UDA functions


### 4.1 Adversarial Discriminator Loss

In [26]:
def get_discriminator_loss(source_pred, target_pred): 
    domain_pred = torch.cat((source_pred, target_pred),dim=0).cuda()
    #print(domain_pred.shape) # [128,1024]
    source_truth = torch.zeros(len(source_pred))
    target_truth = torch.ones(len(target_pred))
    domain_truth = torch.cat((source_truth, target_truth),dim=0).cuda()
    #print(domain_truth.shape) # [128]

    domain_loss = domain_truth*torch.log(1/domain_pred)+(1-domain_truth)*torch.log(1/(1-domain_pred))
    domain_loss = domain_loss.mean()

    return domain_loss 

### 4.2 Adversarial optimizer

In [27]:
def get_adversarial_optimizer(model, config, progress, adversarial=True):
  '''
  Get Adversarial Optimizers
  '''
  lr, wd, momtm = config['lr'], config['wd'], config['momentum']
  lr = lr / ((1 + config['alpha']*progress)**config['beta'])

  feature_ext   = model.get_submodule("feature_extractor")
  classifier    = model.get_submodule("classifier")
  discriminator = model.get_submodule("discriminator")

  pre_trained_weights   = feature_ext.parameters()
  classifier_weights    = classifier.parameters()
  discriminator_weights = discriminator.parameters()

  feature_optim       = torch.optim.SGD([{'params': pre_trained_weights}],     lr=lr/10, weight_decay=wd, momentum=momtm)
  classifier_optim    = torch.optim.SGD([{'params': classifier_weights}],      lr=lr,    weight_decay=wd, momentum=momtm)
  discriminator_optim = torch.optim.SGD([{'params': discriminator_weights}],   lr=lr,    weight_decay=wd, momentum=momtm)
  
  return feature_optim, classifier_optim, discriminator_optim 

### 4.3 Adversarial Train Loop

Setting the **domain adaptation parameter** according to the original [paper](https://arxiv.org/pdf/1505.07818.pdf) section 5.2.2

$$ \lambda_p = \frac{2}{1 + exp(-\gamma \cdot p)} - 1 $$

where p is the training progress linearly changing from 0 to 1.

In [28]:
def adversarial_train_loop(source_loader, target_loader, model, config, progress, device):
  """
  return:
    @best_state
    @best_loss
  """
  size = len(source_loader.dataset)
  
  # cross entropy loss
  classification_loss = get_class_loss_func()

  # Get three optimizer
  feature_optim, class_optim, discriminator_optim = get_adversarial_optimizer(model, config, progress)

  # Target data loader iterator
  iter_target = iter(target_loader)

  domain_adapt = 2 / (1 + math.exp(-config['gamma']*progress)) - 1

  best_loss, best_state = float('inf'), None

  for batch, (X_source, y_source) in enumerate(source_loader):
    try:
      X_target, _ = next(iter_target)
    except:
      iter_target = iter(target_loader)
      X_target, _ = next(iter_target)  

    # Some internal bug return nested tesnor with size 1
    if len(X_source) < 64:
      continue

    X_source, y_source, X_target = X_source.to(device), y_source.to(device), X_target.to(device)

    class_pred_source, domain_pred_source = model(X_source)
    _,                 domain_pred_target = model(X_target)

    class_loss   = classification_loss(class_pred_source, y_source)
    discrim_loss = get_discriminator_loss(domain_pred_source, domain_pred_target)

    feature_optim.zero_grad()

    # Update discriminator
    discriminator_optim.zero_grad()
    discrim_loss.backward(retain_graph=True)
    discriminator_optim.step()

    # Update classifier
    class_optim.zero_grad()
    class_loss.backward(retain_graph=True)
    class_optim.step()

    # Update feature extractor
    feature_optim.step()  

    # Total loss
    total_loss = class_loss - domain_adapt * discrim_loss 

    if total_loss < best_loss:
      best_loss = total_loss
      best_state = model.state_dict()
      print(f"## Update ## best_state updated with loss: {total_loss:>7f}")

    if batch % 10 == 0:
      class_loss, discrim_loss, current = class_loss.item(), discrim_loss.item(), batch * len(X_source)
      total_loss = total_loss.item()
      print(f"## Meter  ## [{current:>5d}/{size:>5d}]")
      # print(f"## Meter  ## classification loss: {class_loss:>7f} discrim loss: {discrim_loss:>7f} total loss: {total_loss:>7f}[{current:>5d}/{size:>5d}]")

    del class_loss, discrim_loss 
    del X_source, y_source, X_target, class_pred_source, domain_pred_source, domain_pred_target
  
  return best_state, best_loss

### 4.4 Adversarial Test Loop

In [29]:
def adversarial_test_loop(dataloader, model, device, name=""):
  test_loss, correct = 0, 0

  class_loss_func = get_class_loss_func()

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      class_pred, _ = model(X)

      test_loss += class_loss_func(class_pred, y).item()
      correct += (class_pred.argmax(1) == y).type(torch.float).sum().item()

  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  test_loss /= num_batches
  correct /= size
  print(f"{name} Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

  return test_loss, correct

### 4.5 Adversarial Training

In [30]:
def adversarial_training(model, source_loader, source_test_loader, target_loader, config, device):
  # print(f"Learning_rate {config['lr']}, weight_decay {config['wd']}")
  best_loss, best_state = float('inf'), None
  no_improve_count = 0

  for epoch in range(config['epochs']):
    print(f"Epoch {epoch+1}\n------------------")
    progress = epoch/config['epochs']

    curr_state, _ = adversarial_train_loop(source_loader, target_loader, model, config, progress, device)

    # Load the best state
    model.load_state_dict(curr_state)

    source_loss, _ = adversarial_test_loop(source_test_loader, model, device, "Source Test")
    target_loss, _ = adversarial_test_loop(target_loader, model, device, "Target Train")

    if target_loss < best_loss:
      no_improve_count = 0

      best_loss = target_loss
      best_state = curr_state
    else:
      no_improve_count += 1

    if no_improve_count > 2:
      print(f"## No Improvement on Target Set, Stopping ##")
      break

  model.load_state_dict(best_state)
  print("Done")

## 5 Training with UDA Techniques

In [31]:
torch.cuda.empty_cache()

In [32]:
adv_model = DANN(len(product_dataset.classes)).to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### 5.1 Product -> Real Life

#### 5.1.1 Training on Product

In [33]:
train_dataloader, train_test_dataloader = get_dataloader(product_dataset, config['batch_size'])
target_dataloader, target_test_dataloader = get_dataloader(real_life_dataset, config['batch_size'])

In [34]:
torch.autograd.set_detect_anomaly(True)
adversarial_training(adv_model, train_dataloader, train_test_dataloader, target_dataloader, config, device)

Epoch 1
------------------
## Update ## best_state updated with loss: 3.002315
## Meter  ## [    0/ 1601]
## Update ## best_state updated with loss: 2.955360
## Update ## best_state updated with loss: 2.949759
## Update ## best_state updated with loss: 2.913088
## Update ## best_state updated with loss: 2.848958
## Update ## best_state updated with loss: 2.832256
## Update ## best_state updated with loss: 2.697971
## Update ## best_state updated with loss: 2.648842
## Update ## best_state updated with loss: 2.480603
## Update ## best_state updated with loss: 2.402117
## Update ## best_state updated with loss: 2.282860
## Meter  ## [  640/ 1601]
## Update ## best_state updated with loss: 2.073895
## Update ## best_state updated with loss: 1.776182
## Update ## best_state updated with loss: 1.662053
## Update ## best_state updated with loss: 1.512952
## Update ## best_state updated with loss: 1.194102
## Update ## best_state updated with loss: 1.004736
## Update ## best_state updated wit

#### 5.1.2 Testing on Real Life

In [35]:
loader_target_dataset = DataLoader(real_life_dataset, batch_size=config['batch_size'], shuffle=False)

test_loop(loader_target_dataset, adv_model, device)

Test Error: 
 Accuracy: 64.6%, Avg loss: 1.243085 



(1.2430853825062513, 0.646)

### 5.2 Real Life -> Product

#### 5.2.1 Training on Real Life

In [36]:
train_dataloader, train_test_dataloader = get_dataloader(real_life_dataset, config['batch_size'])
target_dataloader, target_test_dataloader = get_dataloader(product_dataset, config['batch_size'])

In [37]:
# Training
adversarial_training(adv_model, train_dataloader, train_test_dataloader, target_dataloader, config, device)

Epoch 1
------------------
## Update ## best_state updated with loss: 1.547083
## Meter  ## [    0/ 1601]
## Update ## best_state updated with loss: 1.282004
## Update ## best_state updated with loss: 1.178914
## Update ## best_state updated with loss: 0.928878
## Meter  ## [  640/ 1601]
## Update ## best_state updated with loss: 0.660247
## Meter  ## [ 1280/ 1601]
## Update ## best_state updated with loss: 0.587790
Source Test Test Error: 
 Accuracy: 74.9%, Avg loss: 0.872657 

Target Train Test Error: 
 Accuracy: 91.0%, Avg loss: 0.274368 

Epoch 2
------------------
## Update ## best_state updated with loss: 0.276027
## Meter  ## [    0/ 1601]
## Update ## best_state updated with loss: -0.007978
## Meter  ## [  640/ 1601]
## Update ## best_state updated with loss: -0.037597
## Meter  ## [ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 76.7%, Avg loss: 0.834816 

Target Train Test Error: 
 Accuracy: 90.9%, Avg loss: 0.280146 

Epoch 3
------------------
## Update ## best_state updat

#### 5.2.2 Testing on Product

In [38]:
loader_target_dataset = DataLoader(product_dataset, batch_size=config['batch_size'], shuffle=False)

test_loop(loader_target_dataset, adv_model, device)

Test Error: 
 Accuracy: 91.9%, Avg loss: 0.313109 



(0.31310947152087465, 0.919)

In [39]:
# del source_dataset, train_dataloader, test_dataloader, target_dataset, loader_target_dataset
del adv_model

## 6 Summary

### 6.1 Product -> Real Life


#### 6.1.1 Purely training on Source Domain-Product

Without using any domain adaptation techniques, within 10 epochs training, the classifier network achieves  64.6% accuracy on the target domain.

#### 6.1.2 Training on both Source and Target Domains

Using DANN doamin adaptation technique, the classifier with feature extractor trained on both source and target domain achives 64.6% accuracy, which is the same as the one trained solely on the source domain, with no improvement on accuracy.

### 6.2 Real Life -> Product

#### 6.2.1 Purely Training on Source Domain-Real Life

The classifier trained solely on the source domain achives 79.2% accuracy on the target domain.

#### 6.2.2 Training on both Source and Target Domains

With feature extractor trained on both source and target domain, the classifer achives 91.9% accuracy on the target domain, which is about 12% improvement over feature extractor purely trained on source domain.